In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import (
  HumanMessage,
  SystemMessage,
  ToolMessage
)
from langchain.tools import tool
from langchain_core.output_parsers.openai_tools import parse_tool_calls
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
llm = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.0,
  base_url="https://openai.vocareum.com/v1"
)

Tool Creation

In [3]:
@tool
def multiply(a: int, b:int) -> int:
  """Multiply two numbers."""
  return a * b

In [4]:
tools = [multiply]
tool_map = {tool.name:tool for tool in tools}

In [5]:
tool_map

{'multiply': StructuredTool(name='multiply', description='Multiply two numbers.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x7cc351a50e00>)}

Binding Tools

In [6]:
llm_with_tools = llm.bind_tools(tools)

In [7]:
question = "3 multiplied by 2"

In [8]:
messages = [
  SystemMessage("You're a helpful assistant"),
  HumanMessage(question)
]

In [9]:
ai_message = llm_with_tools.invoke(messages)

In [ ]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GeeXqupi0tmS4XqxLSEQOVeC', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 54, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Blj4MbvlWj5DIqqkYIogxLenZbYoH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b5531fc8-642b-4372-9e3d-ed2df68f624a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_GeeXqupi0tmS4XqxLSEQOVeC', 'type': 'tool_call'}], usage_metadata={'input_tokens': 54, 'output_tokens': 17, 'total_tokens': 71, 'input_token_details': {'audio': 0, 'cach

In [11]:
messages.append(ai_message)
messages

[SystemMessage(content="You're a helpful assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='3 multiplied by 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GeeXqupi0tmS4XqxLSEQOVeC', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 54, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Blj4MbvlWj5DIqqkYIogxLenZbYoH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b5531fc8-642b-4372-9e3d-ed2df68f624a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b

Using Tool Calls

In [13]:
parsed_tool_calls = parse_tool_calls(
  ai_message.additional_kwargs.get("tool_calls")
)

parsed_tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 2},
  'id': 'call_GeeXqupi0tmS4XqxLSEQOVeC',
  'type': 'tool_call'}]

In [14]:
for tool_call in parsed_tool_calls:
  tool_call_id = tool_call['id']
  function_name = tool_call['name']
  arguments = tool_call['args']
  func = tool_map[function_name]
  result = func.invoke(arguments)
  
  tool_message = ToolMessage(
    content=result,
    name=function_name,
    tool_call_id=tool_call_id
  )
  messages.append(tool_message)

Sending the result back to the LLM

In [15]:
messages

[SystemMessage(content="You're a helpful assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='3 multiplied by 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GeeXqupi0tmS4XqxLSEQOVeC', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 54, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Blj4MbvlWj5DIqqkYIogxLenZbYoH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b5531fc8-642b-4372-9e3d-ed2df68f624a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b

In [16]:
ai_message = llm_with_tools.invoke(messages)

In [17]:
ai_message

AIMessage(content='3 multiplied by 2 is 6.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 79, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BljGedblBiDBrGT0gkxCqUZNNrfZz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a28cdef0-68a0-41c8-89f2-d6b44489d3f5-0', usage_metadata={'input_tokens': 79, 'output_tokens': 10, 'total_tokens': 89, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})